<a href="https://colab.research.google.com/github/manvirkaur84/manvirkaur/blob/main/docs/ml-concepts/207_ML_MSBA/SBACaseLogit%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [278]:
%pip install dmba

In [279]:
%matplotlib inline
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pylab as plt
import seaborn as sns
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
import math
from scipy.stats import chi2


DATA = Path('/content/sample_data/')

#***Section A:*** ⤵
####Fit a logistic regression model to reproduce parameter (coefficient) estimates (up to 4 decimals) in Tables 7(a), 8, 9 of this article using the SBA case data SBAcase.11.13.17.csv by using (1) sklearn LogisticRegression() liblinear solver and (2) sklearn LogisticRegression() Default Solver 'lbfgs'.
____________

#Using sklearn LogisticRegression() liblinear solver

##Table 7(a).1

In [280]:
sba_df = pd.read_csv(DATA / 'SBAcase.11.13.17.csv')

# TARGET VARIABLE - 'Default' is our dummy variable derived from "MIS_Status"
# The value for “Default” = 1 if MIS_Status = CHGOFF, and “Default” = 0 if MIS_Status = PIF
sba_df['Default'] = np.where(sba_df['MIS_Status'] == 'CHGOFF', 1, 0)
sba_df.drop(columns=['MIS_Status'], inplace=True)

# Training data only
train = sba_df[sba_df['Selected'] == 1].copy()

# Predictors for Table 7(a)
predictors = ['New', 'RealEstate', 'DisbursementGross', 'Portion', 'Recession']
X_train = train[predictors].copy()
y_train = train['Default']

scaler = StandardScaler()
X_train['DisbursementGross'] = scaler.fit_transform(X_train[['DisbursementGross']])

# Fit logistic regression (liblinear, very large C to mimic no penalty)
logit_reg = LogisticRegression(solver='liblinear', penalty='l2', C=1e42, max_iter=1000)
logit_reg.fit(X_train, y_train)


print("\nTable 7(a) Coefficient Estimates")

train_X2 = sm.add_constant(X_train, prepend=True)
logit_full2 = sm.GLM(y_train, train_X2, family=sm.families.Binomial())
logit_result2 = logit_full2.fit()
logit_result2.summary()

# Build formatted DataFrame
coef_table = pd.DataFrame({
    "Parameter": train_X2.columns,
    "DF": [1]*len(logit_result2.params),
    "Estimate": [f"{val:.4f}" if abs(val) >= 1e-4 else f"{val:.2E}" for val in logit_result2.params],
    "Standard error": [f"{val:.4f}" if abs(val) >= 1e-4 else f"{val:.2E}" for val in logit_result2.bse],
    "Wald Chi-Square": [f"{val:.4f}" for val in (logit_result2.params / logit_result2.bse)**2],
    "Pr > ChiSq": ["<0.0001" if p < 0.0001 else f"{p:.4f}" for p in logit_result2.pvalues]
})

coef_table["Parameter"] = coef_table["Parameter"].replace("const", "Intercept")
print(coef_table.to_string(index=False))



Table 7(a) Coefficient Estimates
        Parameter  DF Estimate Standard error Wald Chi-Square Pr > ChiSq
        Intercept   1   1.2704         0.3430         13.7150     0.0002
              New   1  -0.0772         0.2101          0.1349     0.7134
       RealEstate   1  -2.0331         0.3636         31.2663    <0.0001
DisbursementGross   1  -0.1160         0.1211          0.9173     0.3382
          Portion   1  -2.8298         0.5594         25.5909    <0.0001
        Recession   1   0.4971         0.2413          4.2441     0.0394


##Table 8.1

In [281]:
sba_df = pd.read_csv(DATA / 'SBAcase.11.13.17.csv')

# TARGET VARIABLE - 'Default' is our dummy variable derived from "MIS_Status"
# The value for “Default” = 1 if MIS_Status = CHGOFF, and “Default” = 0 if MIS_Status = PIF
sba_df['Default'] = np.where(sba_df['MIS_Status'] == 'CHGOFF', 1, 0)
sba_df.drop(columns=['MIS_Status'], inplace=True)

# Training data only
train = sba_df[sba_df['Selected'] == 1].copy()

# Predictors for Table 8
predictors = ['RealEstate', 'Portion', 'Recession']
X_train = train[predictors]
y_train = train['Default']

# validation/test set (Selected = 0)
valid_X = sba_df[sba_df['Selected'] == 0][predictors]
valid_y = sba_df[sba_df['Selected'] == 0]['Default']

# Fit logistic regression (liblinear, very large C to mimic no penalty)
logit_reg = LogisticRegression(solver='liblinear', penalty='l2', C=1e42, max_iter=1000)
logit_reg.fit(X_train, y_train)


print("\nTable 7(a) Coefficient Estimates")

train_X2 = sm.add_constant(X_train, prepend=True)
logit_full2 = sm.GLM(y_train, train_X2, family=sm.families.Binomial())
logit_result2 = logit_full2.fit()

# Build formatted DataFrame
coef_table = pd.DataFrame({
    "Parameter": train_X2.columns,
    "DF": [1]*len(logit_result2.params),
    "Estimate": [f"{val:.4f}" if abs(val) >= 1e-4 else f"{val:.2E}" for val in logit_result2.params],
    "Standard error": [f"{val:.4f}" if abs(val) >= 1e-4 else f"{val:.2E}" for val in logit_result2.bse],
    "Wald Chi-Square": [f"{val:.4f}" for val in (logit_result2.params / logit_result2.bse)**2],
    "Pr > ChiSq": ["<0.0001" if p < 0.0001 else f"{p:.4f}" for p in logit_result2.pvalues]
})

coef_table["Parameter"] = coef_table["Parameter"].replace("const", "Intercept")
print(coef_table.to_string(index=False))


Table 7(a) Coefficient Estimates
 Parameter  DF Estimate Standard error Wald Chi-Square Pr > ChiSq
 Intercept   1   1.3931         0.3216         18.7670    <0.0001
RealEstate   1  -2.1282         0.3450         38.0529    <0.0001
   Portion   1  -2.9875         0.5393         30.6898    <0.0001
 Recession   1   0.5041         0.2412          4.3679     0.0366


##Table 9.1

In [282]:
#code from Dicussion Assignment #1  - sklearn LogisticRegression() liblinear solver
#Table 9

# Validation predictions
y_pred = logit_reg.predict(valid_X)

# DMBA classification summary
#classificationSummary(valid_y, y_pred, class_names=[1, 0])

# Confusion matrix
cm = confusion_matrix(valid_y, y_pred)
tn, fp, fn, tp = cm.ravel()

# Build DataFrame with totals
conf_table = pd.DataFrame({
    "Classification": [
        "Higher risk (predicted default)",
        "Lower risk (predicted non-default)",
        "Total"
    ],
    "Loans charged off": [tp, fn, tp+fn],
    "Loans paid in full": [fp, tn, fp+tn],
    "Total": [tp+fp, fn+tn, tp+fn+fp+tn]
})

print(conf_table.to_string(index=False))

accuracy = accuracy_score(valid_y, y_pred)
misclassification_rate = 1 - accuracy

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Misclassification Rate: {misclassification_rate*100:.2f}%")

                    Classification  Loans charged off  Loans paid in full  Total
   Higher risk (predicted default)                 31                  14     45
Lower risk (predicted non-default)                324                 682   1006
                             Total                355                 696   1051
Accuracy: 67.84%
Misclassification Rate: 32.16%


# Using sklearn LogisticRegression() Default Solver **'lbfgs'**

##Table 7(a).2


In [283]:
sba_df = pd.read_csv(DATA / 'SBAcase.11.13.17.csv')

# TARGET VARIABLE - 'Default' is our dummy variable derived from "MIS_Status"
# The value for “Default” = 1 if MIS_Status = CHGOFF, and “Default” = 0 if MIS_Status = PIF
sba_df['Default'] = np.where(sba_df['MIS_Status'] == 'CHGOFF', 1, 0)
sba_df.drop(columns=['MIS_Status'], inplace=True)

# Training data only
train = sba_df[sba_df['Selected'] == 1].copy()

# Predictors for Table 7(a)
predictors = ['New', 'RealEstate', 'DisbursementGross', 'Portion', 'Recession']
X_train = train[predictors].copy()
y_train = train['Default']

# validation/test set (Selected = 0)
valid_X = sba_df[sba_df['Selected'] == 0][predictors]
valid_y = sba_df[sba_df['Selected'] == 0]['Default']

logit_reg_default = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000)
logit_reg_default.fit(X_train, y_train)

print("\nTable 7(a) Coefficient Estimates")

train_X2 = sm.add_constant(X_train, prepend=True)
logit_full2 = sm.GLM(y_train, train_X2, family=sm.families.Binomial())
logit_result2 = logit_full2.fit()

# Build formatted DataFrame
coef_table = pd.DataFrame({
    "Parameter": train_X2.columns,
    "DF": [1]*len(logit_result2.params),
    "Estimate": [f"{val:.4f}" if abs(val) >= 1e-4 else f"{val:.2E}" for val in logit_result2.params],
    "Standard error": [f"{val:.4f}" if abs(val) >= 1e-4 else f"{val:.2E}" for val in logit_result2.bse],
    "Wald Chi-Square": [f"{val:.4f}" for val in (params / logit_result2.bse)**2],
    "Pr > ChiSq": ["<0.0001" if p < 0.0001 else f"{p:.4f}" for p in logit_result2.pvalues]
})

coef_table["Parameter"] = coef_table["Parameter"].replace("const", "Intercept")
print(coef_table.to_string(index=False))


Table 7(a) Coefficient Estimates
        Parameter  DF  Estimate Standard error Wald Chi-Square Pr > ChiSq
        Intercept   1    1.3537         0.3229         17.5729    <0.0001
              New   1   -0.0772         0.2101          0.1349     0.7134
       RealEstate   1   -2.0331         0.3636         31.2663    <0.0001
DisbursementGross   1 -3.37E-07       3.52E-07          0.9173     0.3382
          Portion   1   -2.8298         0.5594         25.5909    <0.0001
        Recession   1    0.4971         0.2413          4.2441     0.0394


##Table 8.2

In [284]:
sba_df = pd.read_csv(DATA / 'SBAcase.11.13.17.csv')

# TARGET VARIABLE - 'Default' is our dummy variable derived from "MIS_Status"
# The value for “Default” = 1 if MIS_Status = CHGOFF, and “Default” = 0 if MIS_Status = PIF
sba_df['Default'] = np.where(sba_df['MIS_Status'] == 'CHGOFF', 1, 0)
sba_df.drop(columns=['MIS_Status'], inplace=True)

# Training data only
train = sba_df[sba_df['Selected'] == 1].copy()

# Predictors for Table 7(a)
predictors = ['RealEstate', 'Portion', 'Recession']
X_train = train[predictors].copy()
y_train = train['Default']

# validation/test set (Selected = 0)
valid_X = sba_df[sba_df['Selected'] == 0][predictors]
valid_y = sba_df[sba_df['Selected'] == 0]['Default']

logit_reg_default = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000)
logit_reg_default.fit(X_train, y_train)

print("\nTable 7(a) Coefficient Estimates")

train_X2 = sm.add_constant(X_train, prepend=True)
logit_full2 = sm.GLM(y_train, train_X2, family=sm.families.Binomial())
logit_result2 = logit_full2.fit()

# Build formatted DataFrame
coef_table = pd.DataFrame({
    "Parameter": train_X2.columns,
    "DF": [1]*len(logit_result2.params),
    "Estimate": [f"{val:.4f}" if abs(val) >= 1e-4 else f"{val:.2E}" for val in logit_result2.params],
    "Standard error": [f"{val:.4f}" if abs(val) >= 1e-4 else f"{val:.2E}" for val in logit_result2.bse],
    "Wald Chi-Square": [f"{val:.4f}" for val in (logit_result2.params / logit_result2.bse)**2],
    "Pr > ChiSq": ["<0.0001" if p < 0.0001 else f"{p:.4f}" for p in logit_result2.pvalues]
})

coef_table["Parameter"] = coef_table["Parameter"].replace("const", "Intercept")
print(coef_table.to_string(index=False))


Table 7(a) Coefficient Estimates
 Parameter  DF Estimate Standard error Wald Chi-Square Pr > ChiSq
 Intercept   1   1.3931         0.3216         18.7670    <0.0001
RealEstate   1  -2.1282         0.3450         38.0529    <0.0001
   Portion   1  -2.9875         0.5393         30.6898    <0.0001
 Recession   1   0.5041         0.2412          4.3679     0.0366


##Table 9.2

In [285]:
# Validation predictions
y_pred = logit_reg.predict(valid_X)

# Then: get confusion matrix values
cm = confusion_matrix(valid_y, y_pred)
tn, fp, fn, tp = cm.ravel() #unpacks values directly from the confusion matrix

# Build textbook-style table with totals
table = pd.DataFrame({
    "Classification": [
        "Higher risk (predicted default)",
        "Lower risk (predicted paid in full)",
        "Total"
    ],
    "Loans charged off": [tp, fn, tp+fn],
    "Loans paid in full": [fp, tn, fp+tn],
    "Total": [tp+fp, fn+tn, tp+fn+fp+tn]
})

print("\nTable 9 California-based scenario: Classification of Loans.")
print(table.to_string(index=False))

accuracy = accuracy_score(valid_y, y_pred)
misclassification_rate = 1 - accuracy

print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Misclassification Rate: {misclassification_rate*100:.2f}%")


Table 9 California-based scenario: Classification of Loans.
                     Classification  Loans charged off  Loans paid in full  Total
    Higher risk (predicted default)                 31                  14     45
Lower risk (predicted paid in full)                324                 682   1006
                              Total                355                 696   1051
Accuracy: 67.84%
Misclassification Rate: 32.16%


#***Section B*** ⤵
####Refer to Table 8 of the article. Write the estimated equation that associates the outcome variable (i.e., default or not) with predictors RealEstate, Portion, and Recession, in three formats:
* The logit as a function of the predictors
* The odds as a function of the predictors
* The probability as a function of the predictors
------


Estimated Equation(s):
* B0 = Intercept
* B2 = RealEstate
* B3 = Portion
* B4 = Recession

p = P(Default = 1 | RealEstate, Portion, Recession)

1) Logit Function = ln(1/(1-p)) = B0 + B1X1 + B2X2 + B3X3
* ln(1/(1-p)) = 1.3931 + (-2.1821 * X1) + (-2.9875 * X2) + (0.5041 * X3)

2) Odds Function = p/(1-p) = e^logit
* e^[1.3931 + (-2.1821 * X1) + (-2.9875 * X2) + (0.5041 * X3)]

3) Probability Function = p = (odds/1+odds)
* [(e^[1.3931 + (-2.1821 * X1) + (-2.9875 * X2) + (0.5041 * X3))/(1+e^[1.3931 + (-2.1821 * X1) + (-2.9875 * X2) + (0.5041 * X3))]

#***Section C*** ⤵
#####Explain why risk indicators in Table 8 were selected using p-values in Table 7(a).
------


Risk indicators in Table 8 were selected using p-values from Table 7(a) to make the model simpler and reliable. Table 7(a) had 5 variables, but DisembursementGross and New were dropped beacaue their p-values were greater than 0.05, which made them statistically insignificant causing them to have no serious effect on the data. The remaining variables all had p-values less than 0.05 and were selected.